In [12]:
from grizly_dev.grizly import Store, QFrame

ui_subquery()

QFrame().build_subquery("store.json")
#QFrame().build_field("store.json")

In [8]:
qf = QFrame().read_json("store.json", subquery="s1")
qf.get_sql()
qf.to_df()

SELECT business_unit_group,
       count(business_unit_group_name) AS count_records
FROM base_views.sales_monthly_close
GROUP BY business_unit_group


,business_unit_group,count_records
0,ENGG,1909808
1,AERG,2233940
2,INDG,3894202
3,MEDG,89902


In [1]:
from grizly import QFrame

In [140]:
import ipywidgets as w
from grizly_dev.grizly import Store, QFrame

class UIField():
    
    def __init__(self, store_path):
        self.store_path = store_path
        self.label = w.Label("Note you need to reload cell for each new field")
        self.field_name = w.Text(value='field_name', description='Field Name:', disabled=False)
        self.expression = w.Textarea(value='SQL Expression Here'
                    , description='Expression:', disabled=False, layout={'width': '100%'})
        self.subquery = w.Text(value='subquery_name', description='Subquery Name:', disabled=False)
        self.type = w.Text(value='dim', description='Type:', disabled=False)
        self._as = w.Text(value='', description='As:', disabled=False)
        self.group_by = w.Text(value='group', description='Group By:', disabled=False)
        self.fields =  w.Dropdown(options=[])
        self.btn_add_to_store = w.Button(description='Add To Store',disabled=False)
        self.btn_update_store = w.Button(description='Update Store',disabled=False)
        self.btn_add_field = w.Button(description='Add Field',disabled=False)
        self.btn_edit_field = w.Button(description='Edit Field',disabled=False)
        self.output = None
        
    def _generate_qf(self):
        qf = QFrame().read_json(self.store_path, subquery=self.subquery.value)
        return qf
        
    def _save_qf(self, qf):
        qf.save_json(self.store_path, subquery=self.subquery.value)
        
    def _btn_add_field(self, button):
        self.label.value = "You are now adding a new field"
        new_out = self.add_field()
        self.output.children += (new_out,)
        
    def _btn_edit_field(self, button):
        self.label.value = "You are now editing"
        self.btn_add_to_store.display = "none"
        new_out = self.edit_field()
        self.output.children += (new_out,)
    
    def _btn_update_store(self, button):
        qf = self._generate_qf()
        try:
            data = qf.data["select"]["fields"][self.field_name.value]
            data["expression"] = self.expression.value
            data["as"] = self.expression._as
            data["type"] = self.expression.type
            data["group_by"] = self.expression.group_by
            data["expression"] = self.expression.value
            self._save_qf(qf)
        except:
            self.label.value = "Error. Are you sure this fields exists?"
        
    def _btn_add_to_store(self, button):
        qf = self._generate_qf()
        qf.assign(**{self.field_name.value:self.expression.value})
        self._save_qf(qf)
    
    def build_field(self):
        row_1 =  w.HBox([self.label])
        row_2 =  w.HBox([self.subquery])
        row_3 =  w.HBox([self.btn_add_field, self.btn_edit_field])
        first_out = w.VBox([row_1, row_2, row_3])
        self.output = first_out
        self.btn_add_field.on_click(self._btn_add_field)
        self.btn_edit_field.on_click(self._btn_edit_field)
        return self.output
        
    def add_field(self):
        row_1 = w.VBox([self.field_name, self.type, self._as, self.group_by])
        row_2 = w.HBox([self.expression])
        row_3 = w.HBox([self.btn_add_to_store])
        self.btn_add_to_store.on_click(self._btn_add_to_store)
        return w.VBox([row_1, row_2, row_3])
    
    def edit_ui(self):
        qf = self._generate_qf()
        field = qf.data["select"]["fields"][self.fields.value]
        self.expression._as = field["as"]
        self.expression.type = field["type"]
        self.expression.group_by = field["group_by"]
        self.expression.value = field["expression"]
    
    def edit_field(self):
        qf = self._generate_qf()
        fields = qf.get_fields()
        self.fields.options = fields
        self.fields.on_trait_change(self.edit_ui, name="value")
        row_1 = w.VBox([self.fields, self.type, self._as, self.group_by])
        row_2 = w.HBox([self.expression])
        row_3 = w.HBox([self.btn_update_store])
        self.edit_ui()
        self.btn_update_store.on_click(self._btn_update_store)
        return w.VBox([row_1, row_2, row_3])
    
ui = UIField("store.json").build_field()
ui

store.json


In [48]:
qf.data["select"]["fields"]["account_manager"]["expression"] = "test"

In [49]:
qf.data

{'select': {'table': 'sales_monthly_close',
  'schema': 'base_views',
  'fields': {'account_manager': {'type': 'dim',
    'as': '',
    'group_by': '',
    'order_by': '',
    'expression': 'test',
    'select': '',
    'custom_type': ''},
   'field': {'type': 'dim',
    'as': 'field',
    'group_by': '',
    'order_by': '',
    'expression': 'some',
    'custom_type': ''},
   'field_name': {'type': 'dim',
    'as': 'field_name',
    'group_by': '',
    'order_by': '',
    'expression': 'some',
    'custom_type': ''},
   'field_nameww': {'type': 'dim',
    'as': 'field_nameww',
    'group_by': '',
    'order_by': '',
    'expression': "case\n\n'fiscal_year' = 2014 then 1\nelse 0\n\nend",
    'custom_type': ''}},
  'engine': 'mssql+pyodbc://redshift_acoe',
  'where': '',
  'distinct': '',
  'having': '',
  'limit': ''}}